# Parte 1 - Ecuacion de difusión en 1D

Una partícula Browniana que se difunde en una caja unidimensional de 100um de ancho, empieza perfectamente localizada en el centro de la caja. La probabilidad de encontrar a la partícula en un punto $x$, después de un tiempo $t$, $P(t,x)$ está dada por la ecuación:

$$
\frac{\partial P(x,t)}{\partial t}=D\frac{\partial^2 P(x,t)}{\partial x^2}
$$

